In [19]:
import re
import json
from datasets import load_dataset, DatasetDict


# —— 工具函数 —— #
def split_sentences(text: str):
    """
    按中文标点切分古诗为独立句子，保留标点。
    """
    text = text.strip()
    if text and text[-1] not in "。！？；":
        text += "。"
    parts = re.split(r'(?<=[。！？；])', text)
    return [p.strip() for p in parts if p.strip()]


# 训练集情感映射（示例，可按实际标签补充）
EMOTION2ID = {
    "爱国": 0,
    "庆祝": 1,
    "咏史": 2,
    "思乡": 3,
    "惆怅与无奈": 4,
    # ……
}


def encode_emotion_train(emotion: str) -> int:
    return EMOTION2ID.get(emotion, -1)


# —— 预处理函数 —— #
def preprocess_train(example):
    """
    训练集样本包含：title, content, keywords(dict), trans, emotion(str)
    输出：
      - input_text: prompt
      - label_text: JSON 字符串，含 ans_qa_words、ans_qa_sents（直接用 trans 拆解）和 choose_id
    """
    title = example["title"].strip()
    content = example["content"].strip()
    keywords = list(example["keywords"].keys())
    trans = example["trans"].strip()
    emotion = example["emotion"].strip()

    # 句子列表：从 content 拆分
    sentences = split_sentences(content)
    # 从 trans 拆分得到对应的白话句列表
    trans_sents = split_sentences(trans)

    # 构造 prompt
    prompt = (
        f"诗题：{title}；内容：{content}；\n"
        f"关键词：{keywords}；\n"
        f"请为每个关键词和以下每一句诗生成白话解释，并判断情感类别。\n"
        f"诗句列表：{sentences}。\n"
        "输出格式（JSON）：{ans_qa_words:…, ans_qa_sents:…, choose_id:…}"
    )
    # 构造 label
    ans_qa_words = {kw: example["keywords"][kw] for kw in keywords}
    ans_qa_sents = {s: t for s, t in zip(sentences, trans_sents)}
    choose_id = encode_emotion_train(emotion)

    label = {
        "ans_qa_words": ans_qa_words,
        "ans_qa_sents": ans_qa_sents,
        "choose_id": choose_id
    }
    return {
        "input_text": prompt,
        "label_text": json.dumps(label, ensure_ascii=False)
    }


def preprocess_val(example):
    """
    验证集样本包含：idx, title, author, content, qa_words(list), qa_sents(list), choose(dict)
    输出：
      - input_text: prompt（与训练一致的格式，供模型推理）
      - qa_words, qa_sents, choose: 原始字段保留，用于后续评估
    """
    title = example["title"].strip()
    content = example["content"].strip()
    qa_words = example["qa_words"]
    qa_sents = example["qa_sents"]
    choose = example["choose"]  # e.g. {"A":"欢快", "B":"无奈", …}

    # 构造 prompt，与训练时一致，只不过不附带答案
    prompt = (
        f"诗题：{title}；内容：{content}；\n"
        f"关键词：{qa_words}；\n"
        f"请为每个关键词和以下每一句诗生成白话解释，并判断情感类别。\n"
        f"诗句列表：{qa_sents}。\n"
        "输出格式（JSON）：{ans_qa_words:…, ans_qa_sents:…, choose_id:…}"
    )
    # 返回输入，以及保留的原始选项和 idx，便于后续解析和评测
    return {
        "idx": example["idx"],
        "input_text": prompt,
        "qa_words": qa_words,
        "qa_sents": qa_sents,
        "choose": choose
    }


# 1. 加载原始训练/验证 JSON
# 假设文件 train.json、val.json 分别为训练、验证集
raw = load_dataset("json", data_files={"train": "./train_data.json"})


FileNotFoundError: Unable to find 'D:\Develop\tianchi\train-data'

In [8]:
raw

DatasetDict({
    train: Dataset({
        features: ['title', 'content', 'keywords', 'trans', 'emotion'],
        num_rows: 164
    })
})

In [11]:
raw["train"]9

{'title': '奉和中书舍人贾至早朝大明宫',
 'content': '鸡鸣紫陌曙光寒，莺啭皇州春色阑。金阙晓钟开万户，玉阶仙仗拥千官。花迎剑佩星初落，柳拂旌旗露未干。独有凤凰池上客，阳春一曲和皆难。',
 'keywords': {'“时时”句': None,
  '一': None,
  '一夜雨': None,
  '一箭风快': None,
  '一轮玉': None,
  '一饷': None,
  '丁未': None,
  '丁未元日': None,
  '丁香': None,
  '万壑松': None,
  '万户': '指皇宫中宫门。',
  '万花川谷': None,
  '万象': None,
  '万里': None,
  '万重山': None,
  '万面鼓声中': None,
  '三十六陂': None,
  '三山': None,
  '三日': None,
  '三更': None,
  '三湘': None,
  '三秦': None,
  '上元': None,
  '上头': None,
  '上将': None,
  '上苑': None,
  '下': None,
  '下邽': None,
  '不解': None,
  '不须': None,
  '且': None,
  '世业': None,
  '世路': None,
  '丙辰岁': None,
  '东君': None,
  '东阳': None,
  '丝丝弄碧': None,
  '丝方尽': None,
  '中书舍人': '官名，时贾至任此职。',
  '中圣': None,
  '中岁': None,
  '中都': None,
  '临': None,
  '丹陛': None,
  '为异客': None,
  '举杯': None,
  '之': None,
  '乌江十五兄': None,
  '乌鸦': None,
  '乍': None,
  '乍窥门户': None,
  '乐': None,
  '乔木': None,
  '乘兴句': None,
  '九十': None,
  '九月九日': None,
  '九派': None,
  '九秋蓬': None,
  '九重': None,
  '习静': None,
 

In [ ]:
# 2. 分别 map 训练/验证预处理
processed = DatasetDict({
    "train": raw["train"].map(preprocess_train, remove_columns=raw["train"].column_names),
    "validation": raw["validation"].map(preprocess_val, remove_columns=raw["validation"].column_names)
})

# 3. 保存到磁盘（可选）
processed.save_to_disk("processed_poetry")
# 或者导出为 JSON lines
processed["train"].to_json("processed_poetry/train.jsonl")
processed["validation"].to_json("processed_poetry/val.jsonl")

print("预处理完成，输出目录：processed_poetry/")

In [ ]:
, "validation": "./eval_data.json"